In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.random_projection import GaussianRandomProjection
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import timm

from sklearn.metrics import accuracy_score
import numpy as np

# Step 1: Define your custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        target = self.targets[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, target

seed = 12
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Step 2: Define transformations for your custom dataset
transform = T.Compose([
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip(),
        T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.10),
        T.RandomAffine(degrees=(-30, 30), translate=(0.2, 0.2), scale=(0.8, 1.2), shear=(-10, 10)),
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])


# Step 3: Define function to get data loaders for train, validation, and test datasets
def get_data_loaders(data_dir, test_dir, batch_size):
    train_data = datasets.ImageFolder(os.path.join(data_dir), transform=transform)
    test_data = datasets.ImageFolder(os.path.join(test_dir), transform=test_transform)

    train_size = int(0.8 * len(train_data))
    val_size = len(train_data) - train_size
    train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

import timm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Step 4: Load pre-trained PyTorch models
model2 = timm.create_model('densenet169', pretrained=True)
model4 = timm.create_model('pvt_v2_b5', pretrained=True)
model3 = timm.create_model('vit_base_patch16_224.orig_in21k_ft_in1k', pretrained=True)
model1 = timm.create_model('deit_base_patch16_224.fb_in1k', pretrained=True)


import torch
import torch.nn as nn

# Function to modify the model
def modify_model(model):
    # Freeze all parameters in the existing model
    for param in model.parameters():
        param.requires_grad = False

    if hasattr(model, 'classifier'):
        # Get the number of input features of the existing classifier layer
        n_inputs = model.classifier.in_features
        # Modify the classifier layer for binary classification
        model.classifier = nn.Sequential(
            nn.Linear(n_inputs, 2048),
            nn.BatchNorm1d(2048),  # Batch Normalization
            nn.ReLU(),
            nn.Dropout(0.2),  # Dropout for regularization

            nn.Linear(2048, 1024),  # Increase the number of neurons
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 6)
        )
    elif hasattr(model, 'head'):
        # Get the number of input features of the existing head layer
        n_inputs = model.head.in_features
        # Modify the head layer for binary classification
        model.head = nn.Sequential(
            nn.Linear(n_inputs, 2048),
            nn.BatchNorm1d(2048),  # Batch Normalization
            nn.ReLU(),
            nn.Dropout(0.2),  # Dropout for regularization

            nn.Linear(2048, 1024),  # Increase the number of neurons
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 6)
        )
    elif hasattr(model, 'fc'):
        # Get the number of input features of the existing fc layer
        n_inputs = model.fc.in_features
        # Modify the fc layer for binary classification
        model.fc = nn.Sequential(
            nn.Linear(n_inputs, 2048),
            nn.BatchNorm1d(2048),  # Batch Normalization
            nn.ReLU(),
            nn.Dropout(0.2),  # Dropout for regularization

            nn.Linear(2048, 1024),  # Increase the number of neurons
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 6)
        )

    return model

# Example usage:
model1 = modify_model(model1)
model2 = modify_model(model2)
model3 = modify_model(model3)
model4 = modify_model(model4)

# Move models to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
models = [model1, model2]#, model3, model4]
for model in models:
    model.to(device)


# Load pre-trained model weights
model_paths = [
    "/kaggle/input/10x-deigset/models/DeiT/model_weights_epoch_199.pth",
    "/kaggle/input/10x-deigset/models/DenseNet169/model_weights_epoch_199_DenseNet169.pth",
    "/kaggle/input/10x-deigset/models/ViT/model_weights_epoch_199_vit.pth",
    "/kaggle/input/10x-deigset/models/pvt/model_weights_epoch_199_pvt.pth"
]

for model, path in zip(models, model_paths):
    model.load_state_dict(torch.load(path))

# Set models to evaluation mode
for model in models:
    model.eval()
# Get data loaders
train_loader, val_loader, test_loader = get_data_loaders("/kaggle/input/10x-diaga-1200/10x/", "/kaggle/input/diag-10x-500/10X/Test", batch_size=128)
# Step 5: Extract features from pre-trained models
def extract_features(model, data_loader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for images, targets in data_loader:
            images = images.to(device)
            features.append(model(images).cpu().numpy())
            labels.append(targets.numpy())
    features = np.concatenate(features)
    labels = np.concatenate(labels)
    return features, labels

train_features = []
val_features = []
test_features = []
train_labels = []
val_labels = []
test_labels = []

for model in models:
    train_feats, train_lbls = extract_features(model, train_loader)
    val_feats, val_lbls = extract_features(model, val_loader)
    test_feats, test_lbls = extract_features(model, test_loader)
    train_features.append(train_feats)
    val_features.append(val_feats)
    test_features.append(test_feats)
    train_labels.append(train_lbls)
    val_labels.append(val_lbls)
    test_labels.append(test_lbls)

# Concatenate features
train_features = np.concatenate(train_features, axis=1)
val_features = np.concatenate(val_features, axis=1)
test_features = np.concatenate(test_features, axis=1)
train_labels = train_labels[0]  # Assuming all models have the same labels
val_labels = val_labels[0]
test_labels = test_labels[0]

# Step 6: Train the ELM classifier
elm_pipeline = make_pipeline(StandardScaler(), GaussianRandomProjection(n_components=1000), MLPClassifier(hidden_layer_sizes=(1000,), activation='relu'))
elm_pipeline.fit(train_features, train_labels)

# Step 7: Evaluate the ELM classifier
train_preds = elm_pipeline.predict(train_features)
val_preds = elm_pipeline.predict(val_features)
test_preds = elm_pipeline.predict(test_features)

train_acc = accuracy_score(train_labels, train_preds)
val_acc = accuracy_score(val_labels, val_preds)
test_acc = accuracy_score(test_labels, test_preds)

print("ELM Classifier Performance:")
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")